In [1]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/fehrdelt/data_ssd/scripts")


from clinical_data.clinical_data import get_clinical_data
from mega_CT_TIQUA.volumes_csv import get_volumes_csv

### 3 Trainings: Clinical data only, Segmentation volumes only, Clinical data + Segmentation volumes

#### Get Clinical data

In [2]:

clinical_data_df = get_clinical_data("/home/fehrdelt/data_ssd/data/clinical_data/cleaned_data.csv")

In [3]:
clinical_data_df.head()

,IPP,age,hemocue_initial,fracas_du_bassin,catecholamines,pression_arterielle_systolique_PAS_arrivee_du_smur,pression_arterielle_diastolique_PAD_arrivee_du_smur,score_glasgow_initial,score_glasgow_moteur_initial,anomalie_pupillaire_prehospitalier,frequence_cardiaque_FC_arrivee_du_smur,arret_cardio_respiratoire_massage,penetrant_objet,ischemie_du_membre,hemorragie_externe,amputation
0,338510.0,79.0,NaN,0.0,0.0,190.0,103.0,15.0,6.0,0.0,137.0,0.0,0.0,0.0,0.0,0.0
1,111247.0,52.0,NaN,0.0,0.0,87.0,49.0,15.0,6.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0
2,72080966.0,23.0,NaN,0.0,0.0,100.0,60.0,15.0,6.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
3,72081026.0,42.0,13.1,0.0,0.0,101.0,64.0,14.0,6.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0
4,72012454.0,34.0,15.8,0.0,0.0,110.0,71.0,15.0,6.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0


#### Get Segmentation volumes

In [4]:
# get the shanoir import csv to link IPP with image name (eg. P0023)
# IPP is column 0, name is column 5
shanoir_import_df = pd.read_csv("/home/fehrdelt/data_ssd/data/clinical_data/shanoir_import.csv", header=None)


In [5]:
shanoir_import_df.head()

,0,1,2,3,4,5,6,7,8,9
0,338510,NaN,NaN,260,FastDiag,P0000,!*inj*,Crane sans IV,2020-08-05,NaN
1,111247,NaN,NaN,260,FastDiag,P0001,!*inj*,Crane sans IV,2020-08-06,NaN
2,72080966,NaN,NaN,260,FastDiag,P0002,!*inj*,Crane sans IV,2020-08-07,NaN
3,72081026,NaN,NaN,260,FastDiag,P0003,!*inj*,Crane sans IV,2020-08-08,NaN
4,72012454,NaN,NaN,260,FastDiag,P0004,!*inj*,Crane sans IV,2020-08-09,NaN


In [8]:
ct_tiqua_temp_directory = "/home/fehrdelt/data_ssd/data/mega_CT_TIQUA_temp/"

volumes = {}

volumes_df = pd.DataFrame(columns=["name", 'supratentorial_IPH', 
                            'supratentorial_SAH', 'supratentorial_Petechiae', 
                            'supratentorial_Edema','infratentorial_IPH', 
                            'infratentorial_SAH', 'infratentorial_Petechiae', 
                            'infratentorial_Edema', 'brainstem_IPH', 
                            'brainstem_SAH', 'brainstem_Petechiae', 
                            'brainstem_Edema','extracerebral_SDH', 'extracerebral_EDH'])

for file in os.listdir(ct_tiqua_temp_directory):
    if os.path.isdir(ct_tiqua_temp_directory+file) and file[0:2] == "P0":
        volumes = get_volumes_csv(ct_tiqua_temp_directory+file+"/"+file+"_custom_nn_ANTS_hist_match_Volumes.csv")
        volumes_df.loc[len(volumes_df)] = [file]+list(volumes.values())


In [9]:
volumes_df.head()

,name,supratentorial_IPH,supratentorial_SAH,supratentorial_Petechiae,supratentorial_Edema,infratentorial_IPH,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,brainstem_Petechiae,brainstem_Edema,extracerebral_SDH,extracerebral_EDH
0,P0001,0,343,0,0,0,15,0,0,0,0,0,0,3,0
1,P0002,0,0,0,0,0,0,0,0,0,0,0,0,104,11685
2,P0003,0,120,0,0,0,0,0,0,0,0,0,0,0,0
3,P0004,0,376,0,0,0,0,0,0,0,0,0,0,183,0
4,P0005,0,19,0,15,0,0,0,0,0,0,0,0,90,0


#### Combine volumes and clinical data in one DataFrame

In [96]:
combined_volumes_clinical_df = pd.DataFrame(columns=["name", 'supratentorial_IPH', 
                            'supratentorial_SAH', 'supratentorial_Petechiae', 
                            'supratentorial_Edema','infratentorial_IPH', 
                            'infratentorial_SAH', 'infratentorial_Petechiae', 
                            'infratentorial_Edema', 'brainstem_IPH', 
                            'brainstem_SAH', 'brainstem_Petechiae', 
                            'brainstem_Edema','extracerebral_SDH', 'extracerebral_EDH',
                            'age', 'hemocue_initial', 'fracas_du_bassin', 'catecholamines', 
                                    'pression_arterielle_systolique_PAS_arrivee_du_smur', 
                                    'pression_arterielle_diastolique_PAD_arrivee_du_smur', 
                                    'score_glasgow_initial', 'score_glasgow_moteur_initial', 
                                    'anomalie_pupillaire_prehospitalier', 'frequence_cardiaque_FC_arrivee_du_smur', 
                                    'arret_cardio_respiratoire_massage', 'penetrant_objet', 'ischemie_du_membre', 
                                    'hemorragie_externe', 'amputation'])


def make_row(id):
    volumes = list(volumes_df.loc[id])
    
    try:
        ipp = (shanoir_import_df[shanoir_import_df[5] == volumes[0]])[0].values[0]
                
    except:
        return
    
    clinical_data = clinical_data_df.loc[clinical_data_df["IPP"]==float(ipp)]

    volumes = np.array(volumes)
    clinical_data = clinical_data.values[0][1:]

    combined = np.concatenate([volumes, clinical_data])


    return combined



for row_id in range(len(volumes_df)):
    
    row = make_row(id=row_id)
    
    combined_volumes_clinical_df.loc[len(combined_volumes_clinical_df)] = row

print(combined_volumes_clinical_df.head)



<bound method NDFrame.head of       name supratentorial_IPH supratentorial_SAH supratentorial_Petechiae  \
0    P0001                  0                343                        0   
1    P0002                  0                  0                        0   
2    P0003                  0                120                        0   
3    P0004                  0                376                        0   
4    P0005                  0                 19                        0   
..     ...                ...                ...                      ...   
191  P0247                  0                168                        0   
192  P0171                 23                 25                        0   
193  P0285                  0               1755                        0   
194  P0210                  0                514                        0   
195  P0251                  0                 42                        0   

    supratentorial_Edema infratentorial_IPH i

In [19]:
print(shanoir_import_df[5])

0      P0000
1      P0001
2      P0002
3      P0003
4      P0004
       ...  
371    P0371
372    P0372
373    P0373
374    P0374
375    P0375
Name: 5, Length: 376, dtype: object
